# Primeiro tu importa o modelo sem o topo. Daí o tensor de saída bate com as deep features.
Tu pode escolher outro tensor de outras formas, mas geralmente galera pega o antes da camada de classificação 


In [1]:
from tensorflow.keras.applications import VGG16
model = VGG16(
    include_top=False,
    weights="imagenet"
)
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

# Executa essa aqui e tu pega da camada que quiser
Eu adiciono uma camada flatten pra sair tudo no formato correto logo

In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import Model

model = VGG16(weights="imagenet")
input_layer = model.input
selected_layer = model.get_layer(index = -5).output
flat_out = layers.Flatten()(selected_layer)

print(input_layer, selected_layer)
model = Model(inputs = input_layer, outputs = flat_out)
model.summary()

Tensor("input_2:0", shape=(None, 224, 224, 3), dtype=float32) Tensor("block5_pool_1/Identity:0", shape=(None, 7, 7, 512), dtype=float32)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     1

In [3]:
import tensorflow as tf
CHANNELS = 3
IMG_HEIGHT = 224
IMG_WIDTH = 224

def get_img(file_path):
	img = tf.io.read_file(file_path)
	img = tf.image.decode_png(img, channels=CHANNELS)
	img = tf.image.convert_image_dtype(img, tf.float32)
	return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

file_pattern = "base/*.jpg"
data = tf.data.Dataset.list_files(file_pattern)
data = data.map(get_img)
data = data.batch(1)

from glob import glob
size = len(glob(file_pattern))
predictions = model.predict(x = data, steps = size, batch_size = 1)

# Um pouco de código que tu pode usar, mas lembrando que tu precisa concatenar a label se tu quiser lembar o que é o que

In [4]:
from numpy import asarray
from numpy import savetxt
import numpy as np
data = asarray(predictions)
header = asarray(range(data.shape[1]))
print(header, data)

[    0     1     2 ... 25085 25086 25087] [[8.8863827e-02 0.0000000e+00 0.0000000e+00 ... 2.7505308e-04
  7.2772437e-01 0.0000000e+00]
 [9.3631789e-02 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  7.1173793e-01 0.0000000e+00]
 [5.4592624e-02 0.0000000e+00 0.0000000e+00 ... 1.1400825e-01
  7.8305143e-01 0.0000000e+00]
 ...
 [7.5190246e-02 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  6.7108810e-01 0.0000000e+00]
 [8.3116561e-02 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  6.7087364e-01 0.0000000e+00]
 [1.1628393e-01 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  6.9747835e-01 0.0000000e+00]]


In [5]:
csv = np.concatenate(([header], data))
print(csv)
savetxt('data.csv', csv, delimiter=',')

[[0.00000000e+00 1.00000000e+00 2.00000000e+00 ... 2.50850000e+04
  2.50860000e+04 2.50870000e+04]
 [8.88638273e-02 0.00000000e+00 0.00000000e+00 ... 2.75053084e-04
  7.27724373e-01 0.00000000e+00]
 [9.36317891e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  7.11737931e-01 0.00000000e+00]
 ...
 [7.51902461e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  6.71088099e-01 0.00000000e+00]
 [8.31165612e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  6.70873642e-01 0.00000000e+00]
 [1.16283931e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  6.97478354e-01 0.00000000e+00]]
